In [37]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import date

###TO DO MAKE THE CODE MORE MODULAR AND REMOVE THE SHITTY MAGIC VARIABLE
### I WILL NOT REFACTOR THIS UNTIL THE MAIN FEATURE IS DONE

def convert_snapshot(data, desired_key = None):
    """
    this extract desired keys from list of dicts from snapshot
    """
    data_dictionary = map(lambda x: x.to_dict(), data)
    if (desired_key is not None):
        data_dictionary = map(lambda x: dict((k, x[k]) for k in desired_key if k in x), list(data_dictionary))

    return list(data_dictionary)


def fill_non_existent_column(df, is_caregiver=False):
    """
    This fill the df with unique problems column i
    """
    if is_caregiver is False:
        new_column = ['ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan',
                      'Gangguan-stres-pascatrauma', 'Skizofrenia']
    else:
        new_column = ['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan',
                      'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']

    for col in new_column:
        if (col in df.columns):
            pass
        else:
            df[col] = 0


def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    """
    A utility method to create a tf.data dataset from a Pandas Dataframe
    """
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds



def age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

def convert_categorical_data(df, col, is_caregiver = False):
    """
    Convert categorical data that splited by space to their own column in pandas dataframe , making them like one hot encoded
    """
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(not is_caregiver):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)


def change_column_name(df, old_name, new_name):
    """
    replace list of column name in old_name to new colum name in new_name
    the order of old_name and new_name need to be exact
    """
    res = {old_name[i]: new_name[i] for i in range(len(new_name))}
    df.rename(columns=res, inplace=True)


def Merge(dict1, dict2):
    """
    Python code to merge dict
    """
    res = {**dict1, **dict2}
    return res


def unpack_caregiver_snapshot(data):
    """

    :param data: data is a snapshot of chatroom collection
    :return: list of caregiver in form of dictionary
    """
    desired_keys = ["problems", "birthday", "gender", "caregiver_id", "text"]
    # this unpack the value of snapshot list and turn the snapshot into dictionary also add aditional key for caregiver_id
    caregiver_dict_unpacked = map(lambda feature: Merge(feature.to_dict(), {'caregiver_id': feature.id}), data)

    # this unpack the snapshot in 'caregiver' snapshot and add them back to main dictionary
    caregiver_dict = map(lambda feature: Merge(feature, feature.pop('caregiver').get().to_dict()),
                         caregiver_dict_unpacked)

    #check for room capacity
    caregiver_dict = filter(lambda feature: feature['max_user'] > len(feature['users']) , caregiver_dict)

    # this seperate desired keys from unused keys
    caregiver_dict = map(lambda x: dict((k, x[k]) for k in desired_keys if k in x), caregiver_dict)

    # this convert birthday stamp into interger age
    caregiver_dict = map(lambda feature: Merge(feature, {'age': age(feature.pop('birthday'))}), caregiver_dict)

    return list(caregiver_dict)

def convert_caregiver_dictList_to_df(dictionary):
    """
    :param dictionary: list of dictionary
    :return: dataframe of the said dictionary
    """
    # Creates DataFrame.
    df = pd.DataFrame(dictionary)
    #change column name to feature according to the model
    change_column_name(df, ['problems','gender','caregiver_id','age'],['Caregiver_Tipe_Masalah','Caregiver_Gender','CAREGIVER_ID','Caregiver_Age'])
    #convert categorical data into their own column
    convert_categorical_data(df, "Caregiver_Tipe_Masalah", is_caregiver = True)
    #fill missing column with default value
    fill_non_existent_column(df, is_caregiver = True)
    return df



In [2]:

from firebase_admin import credentials, firestore, initialize_app
cred = credentials.Certificate("key.json")
default_app = initialize_app(cred)
db = firestore.client()


In [3]:

MAIN_TEXT_MODEL = tf.keras.models.load_model('saved_manual_model/text_query_v1')
MAIN_USER_MODEL = tf.keras.models.load_model('saved_manual_model/user_query_v1')
MAIN_CAREGIVER_MODEL = tf.keras.models.load_model('saved_manual_model/caregiver_query_v1')

In [42]:

db_ref = db.collection('chat_room_pref')
data = db_ref.get()

[{'problems': 'Gangguan-makan ADHD-Hiperaktif-dan-kurang-fokus', 'gender': 'Pria', 'caregiver_id': ' HUDwzgELY1OEzwieoHXK', 'text': 'Depresi bang aku', 'age': 21}]


In [43]:
import tensorflow_recommenders as tfrs
data = unpack_caregiver_snapshot(data)
data = convert_caregiver_dictList_to_df(data)
caregiver_ds = df_to_dataset(data)
TEXT_INDEX = tfrs.layers.factorized_top_k.BruteForce(MAIN_TEXT_MODEL)

                                          problems gender  \
0  Gangguan-makan ADHD-Hiperaktif-dan-kurang-fokus   Pria   

            caregiver_id              text  age  
0   HUDwzgELY1OEzwieoHXK  Depresi bang aku   21  
wtf


In [44]:
TEXT_INDEX = tfrs.layers.factorized_top_k.BruteForce(MAIN_TEXT_MODEL)



INDEX = tfrs.layers.factorized_top_k.BruteForce(MAIN_USER_MODEL)


In [46]:
# update this
TEXT_INDEX.index_from_dataset(
    caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_TEXT_MODEL([features['text']])))
)

# update_this
INDEX.index_from_dataset(
    caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features)))
)

In [45]:
a = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan',
         'Gangguan-stres-pascatrauma', 'Skizofrenia']
value = ['Pria', 30, 0, 1, 0, 0, 1, 0]
dictionary_Test = dict((el, ([value[ix]])) for ix, el in enumerate(a))
print(dictionary_Test)

a, titles = INDEX(dictionary_Test, k=1)
print(f"Top recommendations: {titles[0]}")

a, titles = TEXT_INDEX(np.array(["AAAA sa sayang"]), k=1)

print(f"Top recommendations: {titles[0]}")

print(MAIN_TEXT_MODEL(np.array(["aa sss"])))

{'Gender': ['Pria'], 'Age': [30], 'ADHD-Hiperaktif-dan-kurang-fokus': [0], 'Depresi': [1], 'Gangguan-kecemasan': [0], 'Gangguan-makan': [0], 'Gangguan-stres-pascatrauma': [1], 'Skizofrenia': [0]}
Top recommendations: [b' HUDwzgELY1OEzwieoHXK']
Top recommendations: [b' HUDwzgELY1OEzwieoHXK']
tf.Tensor(
[[ 2.7937052e-01 -2.5666751e-02 -1.3149159e-01 -2.1588609e-01
  -2.2960218e-02 -2.2883387e-01 -1.7346169e-01 -6.4872846e-02
  -2.3486321e-01  7.8157082e-02  2.2771671e-01 -1.0847397e-01
  -1.9453070e-01  3.1591713e-01  1.4165582e-01  5.6409419e-02
   5.1611908e-02  1.4780410e-01  6.2259149e-02  1.6942768e-01
   1.1273560e-01 -1.2807637e-01 -4.7652092e-02  2.5380790e-01
  -1.8733555e-01  1.1730904e-02  1.0061440e-01 -1.2786148e-01
   4.1663740e-03  1.2821472e-01  3.8503781e-01  2.0027758e-01
   1.9084729e-01  7.0396133e-02  9.0815881e-03 -3.2037225e-02
   9.8407986e-03 -8.8083446e-02  1.2190298e-01  4.2459708e-02
  -1.4164943e-01 -4.7461402e-02 -1.7951630e-01 -1.8083632e-05
  -1.3089561e-0

In [77]:
def unpack_user_snapshot(data):
    print(type(data))
    if (type(data) != list):
        data = [data]
    print(type(data))
    data = convert_snapshot(data, desired_key=["problems", "gender", "birthday", "text"])
    data = map(lambda feature: Merge(feature, {'age': age(feature.pop('birthday'))}), data)
    
    return data

def convert_user_dictList_to_df(dictionary):
    """
    :param dictionary: list of dictionary
    :return: dataframe of the said dictionary
    """
    # Creates DataFrame.
    df = pd.DataFrame(dictionary)
    #change column name to feature according to the model
    change_column_name(df, ['problems','gender','caregiver_id','age'],['Tipe_Masalah','Gender','CAREGIVER_ID','Age'])
    #convert categorical data into their own column
    convert_categorical_data(df, "Tipe_Masalah", is_caregiver = False)
    #fill missing column with default value
    fill_non_existent_column(df, is_caregiver = False)
    return df



In [63]:
db_user = db.collection('users').document("4R3TTPINWhPzUvwIUEXbldtNAY72")


In [64]:
data_ref = db_user.get()

In [93]:
data = data_ref
print(data.to_dict())

{'problems': 'Depresi Gangguan-makan', 'name': 'dhiva', 'birthday': DatetimeWithNanoseconds(2000, 11, 16, 16, 0, tzinfo=datetime.timezone.utc), 'text': 'susah makan gara-gara diputusin pacar', 'gender': 'Pria', 'point': 0}


In [94]:
data = list(unpack_user_snapshot(data))
print(data)

<class 'google.cloud.firestore_v1.base_document.DocumentSnapshot'>
<class 'list'>
[{'problems': 'Depresi Gangguan-makan', 'gender': 'Pria', 'text': 'susah makan gara-gara diputusin pacar', 'age': 21}]


In [95]:
data = convert_user_dictList_to_df(data)
data.head(4)

,Gender,text,Age,Depresi,Gangguan-makan,ADHD-Hiperaktif-dan-kurang-fokus,Gangguan-kecemasan,Gangguan-stres-pascatrauma,Skizofrenia
0,Pria,susah makan gara-gara diputusin pacar,21,1,1,0,0,0,0


In [96]:
data =data.to_dict('records')[0]

KeyError: 0

In [98]:
for k, v in data.items():
    data[k] = [v]

In [92]:
a, titles = INDEX(data, k=1)
print(f"Top recommendations: {titles[0]}")

Top recommendations: [b' HUDwzgELY1OEzwieoHXK']


In [100]:
from flask import json

In [117]:
test = {
    'recommendation': titles[0].numpy().tolist()
}

In [118]:
type(test['recommendation'])

list

In [119]:
test = list(map(lambda string: string.decode("utf-8"), test['recommendation']))

In [120]:
test

[' HUDwzgELY1OEzwieoHXK']